# LSTM. 5 featerow - mag, x, y, depth, diff_days

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
SEED = 1337
tf.random.set_seed(SEED)
np.random.seed(SEED)

2023-11-07 17:33:35.029525: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-07 17:33:35.029631: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-07 17:33:35.045578: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-07 17:33:36.441864: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-07 17:33:44.397289: W tensorflow/compiler/

In [2]:
ds_train = tf.data.Dataset.load("../data/ds_train/")
ds_val = tf.data.Dataset.load("../data/ds_val/")
ds_test = tf.data.Dataset.load("../data/ds_test/")

2023-11-07 17:33:52.729694: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-07 17:33:53.788686: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-07 17:33:53.788795: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-07 17:33:53.805081: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-07 17:33:53.805165: I tensorflow/compile

In [3]:
ds_train.cardinality().numpy(), ds_val.cardinality().numpy(), ds_test.cardinality().numpy()

(3222154, 465865, 101377)

In [3]:
BATCH_SIZE = 1024
BLOCK_SIZE = 32

In [4]:
def transform_element(features, labels):
    # transform lebels from (1,) to (2,)
    labels = tf.cast(labels, tf.int32)
    labels = tf.one_hot(labels, depth=2)
    return (features[:,:-1], features[:,-1]), labels[0]

In [5]:
ds_train = ds_train.map(transform_element)
ds_val = ds_val.map(transform_element)
ds_test = ds_test.map(transform_element)

In [6]:
for x, y in ds_train.take(1):
    print(x[0].shape)
    print(x[1].shape)
    print(y.shape)

(32, 4)
(32,)
(2,)


In [7]:
ds_train = ds_train.batch(BATCH_SIZE).shuffle(2000, seed=SEED).prefetch(tf.data.experimental.AUTOTUNE)
ds_val = ds_val.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [8]:
class MyModel(tf.keras.Model):
    def __init__(self, n_embed):
        super(MyModel, self).__init__()
        self.embed = tf.keras.layers.Embedding(100, n_embed // 5)
        self.lstm_embed = tf.keras.layers.LSTM(n_embed // 5 * 4, return_sequences=True)
        self.lstm1 = tf.keras.layers.LSTM(n_embed, return_sequences=True)
        self.lstm2 = tf.keras.layers.LSTM(n_embed)
        self.dense1 = tf.keras.layers.Dense(n_embed, activation='relu')
        self.dense2 = tf.keras.layers.Dense(2)
        self.conc = tf.keras.layers.Concatenate(axis=-1)

    def call(self, inputs):
        cont, disc = inputs
        x1 = self.embed(disc)
        x2 = self.lstm_embed(cont)
        x = self.conc([x1, x2])
        x = self.lstm1(x)
        x = self.lstm2(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return x

In [9]:
model = MyModel(125)

In [13]:
model.build(input_shape=[(None, 32, 4), (None, 32)])
model.count_params()

311502

In [35]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy', tf.keras.metrics.F1Score()])

In [36]:
history = model.fit(ds_train, epochs=20, validation_data=ds_val)

Epoch 1/20


2023-11-03 20:37:17.465716: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:37: Filling up shuffle buffer (this may take a while): 173 of 2000
2023-11-03 20:37:27.479401: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:37: Filling up shuffle buffer (this may take a while): 344 of 2000
2023-11-03 20:37:47.460870: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:37: Filling up shuffle buffer (this may take a while): 706 of 2000
2023-11-03 20:37:57.469777: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:37: Filling up shuffle buffer (this may take a while): 887 of 2000
2023-11-03 20:38:07.474001: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:37: Filling up shuffle buffer (this may take a while): 1078 of 2000
2023-11-03 20:38:17.477980: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:37: Filling up shuffle buffer (this may take a

3190/3190 [==============================] - 339s 69ms/step - loss: 0.2404 - accuracy: 0.9156 - f1_score: 0.6925 - val_loss: 0.2189 - val_accuracy: 0.9263 - val_f1_score: 0.6793
Epoch 2/20


2023-11-03 20:42:51.331198: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:37: Filling up shuffle buffer (this may take a while): 169 of 2000
2023-11-03 20:43:11.335270: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:37: Filling up shuffle buffer (this may take a while): 508 of 2000
2023-11-03 20:43:31.321388: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:37: Filling up shuffle buffer (this may take a while): 886 of 2000
2023-11-03 20:43:51.302328: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:37: Filling up shuffle buffer (this may take a while): 1265 of 2000
2023-11-03 20:44:01.319528: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:37: Filling up shuffle buffer (this may take a while): 1448 of 2000
2023-11-03 20:44:21.336655: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:37: Filling up shuffle buffer (this may take 

3190/3190 [==============================] - 335s 71ms/step - loss: 0.2118 - accuracy: 0.9217 - f1_score: 0.7265 - val_loss: 0.1987 - val_accuracy: 0.9262 - val_f1_score: 0.6804
Epoch 3/20


...

3190/3190 [==============================] - 343s 73ms/step - loss: 0.1895 - accuracy: 0.9266 - f1_score: 0.7473 - val_loss: 0.2062 - val_accuracy: 0.9275 - val_f1_score: 0.6712
Epoch 19/20


2023-11-03 22:21:27.099590: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:37: Filling up shuffle buffer (this may take a while): 165 of 2000
2023-11-03 22:21:37.154074: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:37: Filling up shuffle buffer (this may take a while): 330 of 2000
2023-11-03 22:21:57.143221: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:37: Filling up shuffle buffer (this may take a while): 694 of 2000
2023-11-03 22:22:17.115218: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:37: Filling up shuffle buffer (this may take a while): 1073 of 2000
2023-11-03 22:22:27.116986: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:37: Filling up shuffle buffer (this may take a while): 1264 of 2000
2023-11-03 22:22:37.140549: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:37: Filling up shuffle buffer (this may take 

3190/3190 [==============================] - 347s 75ms/step - loss: 0.1897 - accuracy: 0.9263 - f1_score: 0.7461 - val_loss: 0.2012 - val_accuracy: 0.9275 - val_f1_score: 0.6688
Epoch 20/20


2023-11-03 22:27:14.303034: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:37: Filling up shuffle buffer (this may take a while): 165 of 2000
2023-11-03 22:27:34.319603: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:37: Filling up shuffle buffer (this may take a while): 506 of 2000
2023-11-03 22:27:54.276975: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:37: Filling up shuffle buffer (this may take a while): 880 of 2000
2023-11-03 22:28:04.296728: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:37: Filling up shuffle buffer (this may take a while): 1069 of 2000
2023-11-03 22:28:24.290801: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:37: Filling up shuffle buffer (this may take a while): 1445 of 2000
2023-11-03 22:28:44.297396: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:37: Filling up shuffle buffer (this may take 

3190/3190 [==============================] - 347s 75ms/step - loss: 0.1890 - accuracy: 0.9267 - f1_score: 0.7462 - val_loss: 0.2111 - val_accuracy: 0.9266 - val_f1_score: 0.6859


In [37]:
model.evaluate(ds_test)

100/100 [==============================] - 9s 85ms/step - loss: 0.1299 - accuracy: 0.9579 - f1_score: 0.7631


[0.12989561259746552,
 0.9579293131828308,
 array([0.97793704, 0.5483427 ], dtype=float32)]

In [38]:
model.save("../models/model_v1.keras")

In [39]:
# save history
history = pd.DataFrame.from_dict(history.history)
history.to_csv("../models/history_model_v1.csv", index=False)